In [1]:
import pandas as pd

# The full path to your Excel file
# Using a raw string (r"...") is good practice for Windows paths
file_path = r"D:\Git repo\intelligent_real_estate_assistant\data\Real_Estate_data.xlsx"

In [2]:


# Load each sheet from the Excel file into a separate DataFrame
try:
    active_listings_df = pd.read_excel(file_path, sheet_name='Active_Listings')
    client_database_df = pd.read_excel(file_path, sheet_name='Client_Database')
    archive_df = pd.read_excel(file_path, sheet_name='Archive')

    print("--- Active Listings ---")
    print(active_listings_df.head())
    print("\n--- Active Listings Info ---")
    active_listings_df.info()

    print("\n\n--- Client Database ---")
    print(client_database_df.head())
    print("\n--- Client Database Info ---")
    client_database_df.info()

    print("\n\n--- Archive ---")
    print(archive_df.head())
    print("\n--- Archive Info ---")
    archive_df.info()

except FileNotFoundError:
    print(f"Error: The file was not found at the specified path.")
    print(f"Please double-check the path: {file_path}")
except ValueError as e:
    print(f"Error loading a sheet: {e}")
    print("Please make sure the sheet names ('Active_Listings', 'Client_Database', 'Archive') are correct.")

--- Active Listings ---
     Property ID Listing Status Listing Type Listing Date Building / Society  \
0  SALE-BUNG-101      Available         Sale   2024-10-23       Dada Enclave   
1  RENT-APAR-102      Available         Rent   2024-04-01      Sarna Heights   
2  SALE-BUNG-103      Available         Sale   2024-11-21    Solanki Enclave   
3  SALE-OFFI-104      Available         Sale   2023-11-01         Kala Plaza   
4  RENT-OFFI-105      Available         Rent   2025-06-30          Peri Park   

  Area / Locality            City  Pincode Property Type Bedrooms (BHK)  ...  \
0  Mira Road East  Mira Bhayandar   401107      Bungalow          3 BHK  ...   
1   Shivar Garden  Mira Bhayandar   401107     Apartment          3 BHK  ...   
2    Shanti Nagar  Mira Bhayandar   401105      Bungalow          5 BHK  ...   
3     Golden Nest  Mira Bhayandar   401101  Office Space            NaN  ...   
4    Beverly Park  Mira Bhayandar   401107  Office Space            NaN  ...   

   Property Ag

In [3]:
import re

# --- 1. Clean Column Names ---
# We'll create a function to make this reusable for all dataframes
def clean_col_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = re.sub(r'[^A-Za-z0-9_]+', '', col) # Remove special characters
        new_col = new_col.lower() # Convert to lowercase
        new_cols.append(new_col)
    df.columns = new_cols
    return df

active_listings_df = clean_col_names(active_listings_df)
client_database_df = clean_col_names(client_database_df)
archive_df = clean_col_names(archive_df)

print("--- Cleaned Column Names (Active Listings) ---")
print(active_listings_df.columns)
print("\n")


# --- 2. Process Active Listings DataFrame ---
print("--- Processing Active Listings ---")

# Extract numbers from 'bedroomsbhk' column, converting 'NaN' to 0 for offices etc.
active_listings_df['bedroomsbhk'] = active_listings_df['bedroomsbhk'].astype(str).str.extract('(\d+)').fillna(0).astype(int)

# Rename the cleaned column for clarity
active_listings_df.rename(columns={'bedroomsbhk': 'bhk'}, inplace=True)
print("Cleaned 'bhk' column and handled missing values.")
print(active_listings_df[['propertytype', 'bhk']].head())
print("\n")


# --- 3. Feature Engineering on Client Database ---
print("--- Engineering Features from Client Requirements ---")

# Function to extract BHK preference
def find_bhk(text):
    match = re.search(r'(\d+)\s*BHK', text, re.IGNORECASE)
    return int(match.group(1)) if match else 0

# Function to extract Budget
def find_budget(text):
    match = re.search(r'Budget\s*₹?([\d,]+)L?', text, re.IGNORECASE)
    if match:
        # Remove commas and convert 'L' for Lakhs to actual value
        return int(match.group(1).replace(',', '')) * 100000
    match_rent = re.search(r'Rent\s*up\s*to\s*₹?([\d,]+)', text, re.IGNORECASE)
    if match_rent:
        return int(match_rent.group(1).replace(',', ''))
    return 0

# Function to find property type
def find_property_type(text):
    if re.search(r'BHK', text, re.IGNORECASE):
        return 'Residential'
    if re.search(r'Office|Shop', text, re.IGNORECASE):
        return 'Commercial'
    return 'Any'

# Function to find furnishing status
def find_furnishing(text):
    if re.search(r'unfurnished', text, re.IGNORECASE):
        return 'Unfurnished'
    if re.search(r'semi-furnished', text, re.IGNORECASE):
        return 'Semi-Furnished'
    if re.search(r'furnished', text, re.IGNORECASE):
        return 'Furnished'
    return 'Any'


# Apply the functions to create new structured columns
client_database_df['req_bhk'] = client_database_df['requirements'].apply(find_bhk)
client_database_df['req_budget'] = client_database_df['requirements'].apply(find_budget)
client_database_df['req_property_type'] = client_database_df['requirements'].apply(find_property_type)
client_database_df['req_furnishing'] = client_database_df['requirements'].apply(find_furnishing)

print("Created new structured columns from 'requirements' text.")

print("\n\n--- Final Client Database Head ---")
print(client_database_df[['requirements', 'req_bhk', 'req_budget', 'req_property_type', 'req_furnishing']].head())

print("\n\n--- Final Client Database Info ---")
client_database_df.info()


--- Cleaned Column Names (Active Listings) ---
Index(['propertyid', 'listingstatus', 'listingtype', 'listingdate',
       'buildingsociety', 'arealocality', 'city', 'pincode', 'propertytype',
       'bedroomsbhk', 'bathrooms', 'areasqft', 'areatype', 'floornumber',
       'totalfloors', 'furnishing', 'facingdirection', 'parkingcars',
       'propertyageyrs', 'amenities', 'askingprice', 'monthlyrent',
       'securitydeposit', 'maintmonth', 'pricenegotiable', 'commission',
       'ownername', 'ownerphone'],
      dtype='object')


--- Processing Active Listings ---
Cleaned 'bhk' column and handled missing values.
   propertytype  bhk
0      Bungalow    3
1     Apartment    3
2      Bungalow    5
3  Office Space    0
4  Office Space    0


--- Engineering Features from Client Requirements ---
Created new structured columns from 'requirements' text.


--- Final Client Database Head ---
                                        requirements  req_bhk  req_budget  \
0  3 BHK Semi-Furnished in 

In [4]:

print("--- Fixing the Budget Extraction Function ---")

# The original function had trouble with the 'â‚¹' symbol.
# This revised version looks for 'Budget' followed by ANY non-digit characters, then the numbers.
# This makes it much more robust to encoding issues.
def find_budget_corrected(text):
    # Regex for Sale budget (e.g., "Budget â‚¹85L" or "Budget ₹8500000")
    match = re.search(r'Budget[^\d]*([\d,]+)L?', text, re.IGNORECASE)
    if match:
        # Remove commas, convert 'L' (Lakhs) to a full number
        return int(match.group(1).replace(',', '')) * 100000

    # Regex for Rent budget (e.g., "Rent up to â‚¹45,000")
    match_rent = re.search(r'Rent[^\d]*([\d,]+)', text, re.IGNORECASE)
    if match_rent:
        return int(match_rent.group(1).replace(',', ''))
        
    return 0 # Return 0 if no budget is found

# --- Re-apply the corrected function ---
client_database_df['req_budget'] = client_database_df['requirements'].apply(find_budget_corrected)

print("Budget extraction function has been fixed and re-applied.")
print("The 'req_budget' column should now be correct.")


# --- Display the corrected DataFrame ---
print("\n\n--- Corrected Client Database Head ---")
print(client_database_df[['requirements', 'req_bhk', 'req_budget', 'req_property_type', 'req_furnishing']].head())

--- Fixing the Budget Extraction Function ---
Budget extraction function has been fixed and re-applied.
The 'req_budget' column should now be correct.


--- Corrected Client Database Head ---
                                        requirements  req_bhk  req_budget  \
0  3 BHK Semi-Furnished in Bhayandar East, Rent u...        3          48   
1  3 BHK Unfurnished in Mira Road East, Rent up t...        3          50   
2             2 BHK in Bhayandar West, Budget â‚¹85L        2     8500000   
3  2 BHK in Anywhere in Mira Bhayandar, Budget â‚...        2    14000000   
4            2 BHK in Mira Road East, Budget â‚¹125L        2    12500000   

  req_property_type  req_furnishing  
0       Residential  Semi-Furnished  
1       Residential     Unfurnished  
2       Residential             Any  
3       Residential             Any  
4       Residential             Any  


In [5]:
# Let's make sure our dataframes are available for the function
# In a real application, you'd load these once. In a notebook, we just need to ensure the variables exist.
# No need to re-run the loading and cleaning code, just this block.

def find_matching_properties(client_id):
    """
    Finds matching properties for a given client ID from the database.
    """
    # --- 1. Get Client Requirements ---
    try:
        client_data = client_database_df[client_database_df['clientid'] == client_id].iloc[0]
    except IndexError:
        return f"Client with ID {client_id} not found."

    print(f"--- Finding Match for Client: {client_data['clientname']} ({client_id}) ---")
    print(f"Requirements: {client_data['requirements']}\n")

    # --- 2. Filter Listings Based on Core Criteria ---
    # Start with all active listings
    potential_matches = active_listings_df.copy()

    # Filter by Listing Type (Rent/Sale)
    potential_matches = potential_matches[potential_matches['listingtype'].str.lower() == client_data['lookingfor'].lower()]

    # Filter by BHK - must be at least what the client wants
    potential_matches = potential_matches[potential_matches['bhk'] >= client_data['req_bhk']]

    # --- 3. Filter by Budget ---
    budget = client_data['req_budget']
    # Allow for a 10% flexibility above the budget
    budget_ceiling = budget * 1.10

    if client_data['lookingfor'].lower() == 'sale':
        # For 'Sale', filter by 'askingprice'
        final_matches = potential_matches[potential_matches['askingprice'] <= budget_ceiling]
    elif client_data['lookingfor'].lower() == 'rent':
        # For 'Rent', filter by 'monthlyrent'
        final_matches = potential_matches[potential_matches['monthlyrent'] <= budget_ceiling]
    else:
        # If lookingfor is not sale or rent, return empty
        final_matches = pd.DataFrame()


    # --- 4. Return Top 5 Matches ---
    # Select most relevant columns to display
    display_columns = ['propertyid', 'arealocality', 'propertytype', 'bhk', 'areasqft', 'furnishing', 'askingprice', 'monthlyrent']
    
    if final_matches.empty:
        return "No matching properties found for this client."

    return final_matches[display_columns].head(5)


# --- Let's Test Our Function! ---
# We'll use client CL-1003 (Budget ₹85L) as our test case.
test_client_id = 'CL-1003'
recommended_properties = find_matching_properties(test_client_id)

# Display the recommendations
print("--- Top 5 Recommended Properties ---")
print(recommended_properties)

--- Finding Match for Client: Bhavani Keer (CL-1003) ---
Requirements: 2 BHK in Bhayandar West, Budget â‚¹85L

--- Top 5 Recommended Properties ---
        propertyid    arealocality propertytype  bhk  areasqft  \
20   SALE-APAR-121   Shivar Garden    Apartment    2       817   
84   SALE-APAR-185     Golden Nest    Apartment    2       844   
416  SALE-APAR-517  Mira Road East    Apartment    2       756   
596  SALE-APAR-697      Jesal Park    Apartment    2       708   
621  SALE-APAR-722      Jesal Park    Apartment    2       894   

          furnishing  askingprice  monthlyrent  
20   Fully Furnished    9300000.0          NaN  
84       Unfurnished    9000000.0          NaN  
416   Semi-Furnished    8500000.0          NaN  
596      Unfurnished    8100000.0          NaN  
621   Semi-Furnished    8100000.0          NaN  


In [6]:
# --- 1. First, we need to extract the location from the requirements ---
print("--- Creating Location Extraction Function ---")

def find_location(text):
    """Extracts the area/locality from the requirement text."""
    # This regex looks for "in" followed by words, stopping at a comma or the end of the string.
    match = re.search(r'\bin\s+([\w\s]+)', text, re.IGNORECASE)
    if match:
        location = match.group(1).strip()
        # Handle the "Anywhere in" case specifically
        if 'anywhere' in location.lower():
            return 'Any'
        return location
    return 'Any' # Default if no specific location is found

# Apply this to our client database to create the new 'req_locality' column
client_database_df['req_locality'] = client_database_df['requirements'].apply(find_location)

print("Created 'req_locality' column in the client database.")
print(client_database_df[['requirements', 'req_locality']].head())


# --- 2. Now, let's upgrade our main recommendation function ---
print("\n--- Upgrading the Recommendation Function with Location Filter ---")

def find_matching_properties_v2(client_id):
    """
    Finds matching properties for a client, NOW WITH LOCATION FILTERING.
    """
    try:
        client_data = client_database_df[client_database_df['clientid'] == client_id].iloc[0]
    except IndexError:
        return f"Client with ID {client_id} not found."

    print(f"--- Finding Match for Client: {client_data['clientname']} ({client_id}) ---")
    print(f"Requirements: {client_data['requirements']}\n")

    potential_matches = active_listings_df.copy()

    # -- NEW: Filter by Location FIRST --
    client_locality = client_data['req_locality']
    if client_locality != 'Any':
        print(f"Filtering by Locality: {client_locality}")
        # Use .str.contains() to allow for partial matches (e.g., 'Bhayandar West' in 'Bhayandar West')
        potential_matches = potential_matches[potential_matches['arealocality'].str.contains(client_locality, case=False)]

    # Filter by Listing Type (Rent/Sale)
    potential_matches = potential_matches[potential_matches['listingtype'].str.lower() == client_data['lookingfor'].lower()]
    # Filter by BHK
    potential_matches = potential_matches[potential_matches['bhk'] >= client_data['req_bhk']]

    # Filter by Budget
    budget = client_data['req_budget']
    budget_ceiling = budget * 1.10 # 10% flexibility
    if client_data['lookingfor'].lower() == 'sale':
        final_matches = potential_matches[potential_matches['askingprice'] <= budget_ceiling]
    else: # Rent
        final_matches = potential_matches[potential_matches['monthlyrent'] <= budget_ceiling]

    # --- Return Top 5 Matches ---
    display_columns = ['propertyid', 'arealocality', 'propertytype', 'bhk', 'askingprice', 'monthlyrent']

    if final_matches.empty:
        return "No matching properties found for this client with the specified criteria."

    return final_matches[display_columns].head(5)


# --- Let's Test the Upgraded Function! ---
test_client_id = 'CL-1003'
recommended_properties = find_matching_properties_v2(test_client_id)

print("\n--- Top 5 Recommended Properties (Location-Aware) ---")
print(recommended_properties)

--- Creating Location Extraction Function ---
Created 'req_locality' column in the client database.
                                        requirements    req_locality
0  3 BHK Semi-Furnished in Bhayandar East, Rent u...  Bhayandar East
1  3 BHK Unfurnished in Mira Road East, Rent up t...  Mira Road East
2             2 BHK in Bhayandar West, Budget â‚¹85L  Bhayandar West
3  2 BHK in Anywhere in Mira Bhayandar, Budget â‚...             Any
4            2 BHK in Mira Road East, Budget â‚¹125L  Mira Road East

--- Upgrading the Recommendation Function with Location Filter ---
--- Finding Match for Client: Bhavani Keer (CL-1003) ---
Requirements: 2 BHK in Bhayandar West, Budget â‚¹85L

Filtering by Locality: Bhayandar West

--- Top 5 Recommended Properties (Location-Aware) ---
No matching properties found for this client with the specified criteria.


In [7]:
def get_recommendations(client_id):
    """
    Finds matching properties with a fallback strategy.
    V1: Tries a perfect match with location.
    V2: If no results, it ignores location to find other matches.
    """
    try:
        client_data = client_database_df[client_database_df['clientid'] == client_id].iloc[0]
    except IndexError:
        return f"Client with ID {client_id} not found."

    print(f"--- Finding Match for Client: {client_data['clientname']} ({client_id}) ---")
    print(f"Requirements: {client_data['requirements']}\n")

    # --- BASE FILTERS (APPLIES TO BOTH ATTEMPTS) ---
    base_matches = active_listings_df.copy()
    # Filter by Listing Type
    base_matches = base_matches[base_matches['listingtype'].str.lower() == client_data['lookingfor'].lower()]
    # Filter by BHK
    base_matches = base_matches[base_matches['bhk'] >= client_data['req_bhk']]
    # Filter by Budget
    budget = client_data['req_budget']
    budget_ceiling = budget * 1.10
    if client_data['lookingfor'].lower() == 'sale':
        base_matches = base_matches[base_matches['askingprice'] <= budget_ceiling]
    else: # Rent
        base_matches = base_matches[base_matches['monthlyrent'] <= budget_ceiling]


    # --- ATTEMPT 1: Strict search with location ---
    print("--> Attempt 1: Searching for a perfect match (including location)...")
    client_locality = client_data['req_locality']
    final_matches = pd.DataFrame() # Initialize an empty DataFrame

    if client_locality != 'Any':
        strict_matches = base_matches[base_matches['arealocality'].str.contains(client_locality, case=False)]
        if not strict_matches.empty:
            final_matches = strict_matches

    # --- ATTEMPT 2: Fallback if no strict matches were found ---
    if final_matches.empty:
        print("--> No exact matches found. Expanding search to other localities...")
        final_matches = base_matches # Use the base-filtered list


    # --- Return Top 5 Results ---
    display_columns = ['propertyid', 'arealocality', 'propertytype', 'bhk', 'askingprice', 'monthlyrent']

    if final_matches.empty:
        return "No matching properties found even after expanding the search."

    return final_matches[display_columns].head(5)


# --- Let's Test the Final, Smart Function! ---
test_client_id = 'CL-1003'
recommended_properties = get_recommendations(test_client_id)

print("\n--- Top 5 Recommended Properties (with Fallback Logic) ---")
print(recommended_properties)

--- Finding Match for Client: Bhavani Keer (CL-1003) ---
Requirements: 2 BHK in Bhayandar West, Budget â‚¹85L

--> Attempt 1: Searching for a perfect match (including location)...
--> No exact matches found. Expanding search to other localities...

--- Top 5 Recommended Properties (with Fallback Logic) ---
        propertyid    arealocality propertytype  bhk  askingprice  monthlyrent
20   SALE-APAR-121   Shivar Garden    Apartment    2    9300000.0          NaN
84   SALE-APAR-185     Golden Nest    Apartment    2    9000000.0          NaN
416  SALE-APAR-517  Mira Road East    Apartment    2    8500000.0          NaN
596  SALE-APAR-697      Jesal Park    Apartment    2    8100000.0          NaN
621  SALE-APAR-722      Jesal Park    Apartment    2    8100000.0          NaN


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

print("--- Preparing Data for Price Prediction Model (Corrected) ---")

# --- Step 1: Perform the missing cleaning step on archive_df ---
# Extract numbers from 'bedroomsbhk' column, converting 'NaN' to 0
archive_df['bhk'] = archive_df['bedroomsbhk'].astype(str).str.extract(r'(\d+)').fillna(0).astype(int)
print("Cleaned 'bhk' column in the archive dataframe.")

# --- Step 2: Select features and target ---
# We now use the cleaned 'bhk' and have removed 'bathrooms'.
sales_df = archive_df[archive_df['listingtype'] == 'Sale'].copy()

features = ['arealocality', 'bhk', 'areasqft'] # Corrected feature list
target = 'finalprice'

model_df = sales_df[features + [target]]
model_df.dropna(subset=[target], inplace=True)

print("\nOriginal data shape for sales:", model_df.shape)
print(model_df.head())


# --- Step 3: One-Hot Encoding ---
# This part remains the same but will now work correctly.
X = pd.get_dummies(model_df[features], columns=['arealocality'], drop_first=True)
y = model_df[target]

print("\n--- Model-Ready Data (after One-Hot Encoding) ---")
print("Shape of our features (X):", X.shape)
print("Shape of our target (y):", y.shape)
print(X.head())

--- Preparing Data for Price Prediction Model (Corrected) ---
Cleaned 'bhk' column in the archive dataframe.

Original data shape for sales: (514, 4)
     arealocality  bhk  areasqft  finalprice
1    Shanti Nagar    0       882    19146491
3  Bhayandar West    2       828     9821493
4    Shanti Nagar    3      1773    18500624
5  Bhayandar West    0      2075    38642781
9    Shanti Nagar    1       502     5886655

--- Model-Ready Data (after One-Hot Encoding) ---
Shape of our features (X): (514, 8)
Shape of our target (y): (514,)
   bhk  areasqft  arealocality_Bhayandar East  arealocality_Bhayandar West  \
1    0       882                        False                        False   
3    2       828                        False                         True   
4    3      1773                        False                        False   
5    0      2075                        False                         True   
9    1       502                        False                        Fa

C:\Users\vikas\AppData\Local\Temp\ipykernel_4372\1616440384.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df.dropna(subset=[target], inplace=True)


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

print("--- Step 9: Training the Price Prediction Model ---")

# We already have our features 'X' and target 'y' from the previous step.

# --- 1. Split the data into training and testing sets ---
# 80% of the data will be for training, 20% for testing.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]} properties")
print(f"Testing set size: {X_test.shape[0]} properties")


# --- 2. Initialize and Train the Model ---
# We use RandomForestRegressor, a powerful and reliable model.
# random_state=42 ensures we get the same result every time we run this.
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

print("\nTraining the Random Forest model...")
model.fit(X_train, y_train)
print("Model training complete.")


# --- 3. Make Predictions on the Test Set ---
print("\nMaking predictions on the unseen test data...")
y_pred = model.predict(X_test)


# --- 4. Evaluate the Model's Performance ---
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n--- Model Evaluation Results ---")
print(f"R-squared (R²) Score: {r2:.2f}")
print(f"Mean Absolute Error (MAE): ₹{mae:,.0f}")

print("\n--- Interpretation ---")
print(f"Our model can explain {r2:.0%} of the variance in the property prices.")
print(f"On average, the model's price predictions are off by about ₹{mae:,.0f}.")

# Let's look at a few sample predictions vs actual prices
print("\n--- Sample Predictions vs. Actual Prices ---")
sample_results = pd.DataFrame({'Actual Price': y_test, 'Predicted Price': y_pred}).head(5)
sample_results['Predicted Price'] = sample_results['Predicted Price'].apply(lambda x: f"₹{x:,.0f}")
sample_results['Actual Price'] = sample_results['Actual Price'].apply(lambda x: f"₹{x:,.0f}")
print(sample_results)

--- Step 9: Training the Price Prediction Model ---
Training set size: 411 properties
Testing set size: 103 properties

Training the Random Forest model...
Model training complete.

Making predictions on the unseen test data...

--- Model Evaluation Results ---
R-squared (R²) Score: 0.08
Mean Absolute Error (MAE): ₹6,896,950

--- Interpretation ---
Our model can explain 8% of the variance in the property prices.
On average, the model's price predictions are off by about ₹6,896,950.

--- Sample Predictions vs. Actual Prices ---
    Actual Price Predicted Price
606   ₹6,043,028      ₹5,271,861
971  ₹18,646,234     ₹19,994,830
870  ₹19,604,344     ₹16,133,221
311   ₹9,204,466     ₹15,142,704
974  ₹13,073,840     ₹12,873,109


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd

print("--- Step 10: Retraining Model with an Additional Feature ('propertytype') ---")

# --- 1. Prepare Data with the New Feature ---
sales_df = archive_df[archive_df['listingtype'] == 'Sale'].copy()

# Add 'propertytype' to our list of features
features = ['arealocality', 'propertytype', 'bhk', 'areasqft']
target = 'finalprice'

model_df = sales_df[features + [target]].copy()
model_df.dropna(subset=[target], inplace=True)

# One-hot encode BOTH 'arealocality' and 'propertytype'
X = pd.get_dummies(model_df[features], columns=['arealocality', 'propertytype'], drop_first=True)
y = model_df[target]

# --- 2. Split, Train, and Evaluate ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]} properties")
print(f"Testing set size: {X_test.shape[0]} properties")
print(f"Total features after encoding: {X_train.shape[1]}")

model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
print("\nRetraining the Random Forest model with more features...")
model.fit(X_train, y_train)
print("Model training complete.")

y_pred = model.predict(X_test)

# --- 3. Evaluate the New Model's Performance ---
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("\n--- New Model Evaluation Results ---")
print(f"New R-squared (R²) Score: {r2:.2f}")
print(f"New Mean Absolute Error (MAE): ₹{mae:,.0f}")

print("\n--- Interpretation ---")
print(f"Our new model can explain {r2:.0%} of the variance in the property prices.")
print(f"On average, the model's predictions are now off by about ₹{mae:,.0f}.")

--- Step 10: Retraining Model with an Additional Feature ('propertytype') ---
Training set size: 411 properties
Testing set size: 103 properties
Total features after encoding: 10

Retraining the Random Forest model with more features...
Model training complete.

--- New Model Evaluation Results ---
New R-squared (R²) Score: 0.10
New Mean Absolute Error (MAE): ₹6,930,401

--- Interpretation ---
Our new model can explain 10% of the variance in the property prices.
On average, the model's predictions are now off by about ₹6,930,401.


In [13]:
import joblib
import json

print("--- Step 11: Saving the Model and Required Columns ---")

# --- 1. Save the Trained Model ---
# We are saving the last model we trained
model_filename = 'price_predictor_model.joblib'
joblib.dump(model, model_filename)

print(f"Model saved successfully as '{model_filename}'")


# --- 2. Save the Model's Columns ---
# It's crucial to save the exact column order and names after one-hot encoding
# This ensures that when we get new data, we can make it look identical to what the model was trained on.
model_columns = list(X_train.columns)
columns_filename = 'model_columns.json'
with open(columns_filename, 'w') as f:
    json.dump(model_columns, f)

print(f"Model columns saved successfully as '{columns_filename}'")

--- Step 11: Saving the Model and Required Columns ---
Model saved successfully as 'price_predictor_model.joblib'
Model columns saved successfully as 'model_columns.json'
